In [1]:
import requests
import pandas as pd
import datetime as dt
import io
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools

%matplotlib inline 
init_notebook_mode(connected=True)

### Задача №2

Построить все значения Владимирского индекса топлива для реактивных двигателей (SPI_TRD) и все значения Индекса МАУ (MAU_TRD) на одном графике (по дням) с одновременным отображением объемов (в тоннах)

In [2]:
# URL значений индексов за всю историю
CSV_SPI = 'http://spimex.com/indexes/service_functions.php?request=csv&index=spi'

# URL значений индекса МАУ за всю историю  
CSV_EBP = 'http://spimex.com/indexes/service_functions.php?request=csv&index=ebp'

In [3]:
def upload_csv(csv_url):
    """Загружает, обрабатывает и возвращает csv-файл."""
    download=requests.get(csv_url).content
    csv=pd.read_csv(io.StringIO(download.decode('utf-8')), header=0, sep=';',
                   converters={"volume":float, "index_value":float})
    return csv    

In [4]:
spi = upload_csv(CSV_SPI)
ebp = upload_csv(CSV_EBP)

In [5]:
spi = spi[spi.goods_code == 'TRD']

In [6]:
merged = pd.merge(spi, ebp, on=['date', 'goods_code'], 
                  how='outer', suffixes=('_spi', '_ebp'))
merged['date'] = pd.to_datetime(merged['date'], format='%Y-%m-%d')

Для построения требуемого графика воспользуемся plot.ly:

In [9]:
dates = merged['date']
index_value_spi = merged['index_value_spi']
index_value_ebp = merged['index_value_ebp']
volume_spi = merged['volume_spi']
volume_ebp = merged['volume_ebp']

trace_spi = go.Scatter(
    x = dates,
    y = index_value_spi,
    mode = 'lines',
    name = u'SPI_TRD_INDEX',
)

trace_ebp = go.Scatter(
    x = dates,
    y = index_value_ebp,
    mode = 'lines',
    name = u'MAU_TRD_INDEX',
)

trace_spi_volumes = go.Bar(
    x = dates,
    y = volume_spi,
    name = u'SPI_TRD_VOLUME',
)

trace_ebp_volumes = go.Bar(
    x = dates,
    y = volume_ebp,
    name = u'MAU_TRD_VOLUME',
)


layout = go.Layout(
    title='SPI_TRD & MAU_TRD',
)

fig = go.Figure(data =\
        [trace_spi, trace_ebp,trace_spi_volumes, trace_ebp_volumes],
        layout = layout)
iplot(fig)